In [ ]:
#!/usr/bin/env python3
# -*- encoding: utf-8 -*-
# @Directory: D:\Document\local_rps\python3\scripts
# @File     : psm_demo.ipynb
# @Time     : 2022/01/28 11:25:45
# @Author   : songpeiyao 
# @Version  : 1.0
# @Contact  : ppppy161@qq.com
# @Desc     : None


import warnings 
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
%matplotlib inline
import os
from scipy import stats
import matplotlib.pyplot as plt
import patsy
import sys
from statsmodels.genmod.generalized_linear_model import GLM
import statsmodels.api as sm 
import seaborn as sns


In [ ]:
# 1. 读取文件，数据处理方便被模型计算的形式
# 2. 

In [ ]:
path = os.getcwd() # 获取当前的路径
file = path + "/data.txt"  # 写具体的引用文件名，该文件数据包括所有的X变量和Y变量（0，1）
df = pd.read_csv(file,sep='\t')  # 结合具体文件类型，若为txt文件则用read_csv, excel 则用read_excel

X_field = ['X1','X2','X3','X4','X5','X6'] 
Y_field = ['Y']
field = X_field + Y_field

data = df[field]
data = data.dropna()   # 去除任何包含无效数据的行
treated = data[data['Y']==1]
control = data[data['Y']==0]
treated[field].describe().round(2)   # PSM前对照组和实验组的差异
control[field].describe().round(2)

data.groupby(Y_field).size()  # 原数据样本量大小
treated_sample = treated.sample(10000,axis=0)[field].reset_index(drop=True)  # 实验组
control_sample = control.sample(100000,axis=0)[field].reset_index(drop=True)  # 对照组，数据量必须大于实验组数据量


In [ ]:
data_p = control_sample.append(treated_sample).reset_index(drop=True)  # 两个筛选的数据集为回归基础数据
y_f, x_f = patsy.dmatrices('{} ~ {}'.format(Y_field[0], '+'.join(X_field)), data=data_p, return_type='dataframe')
formula = '{} ~ {}'.format(Y_field[0], '+'.join(X_field))
print('Formula:\n'+formula)
print('n majority:', len(control_sample))
print('n minority:', len(treated_sample))  

# 确定回归方程
i=0   
nmodels=50  # 可指定归回模型个数  应该叫循环次数
errors=0
model_accuracy = []
models = []
while i < nmodels and errors < 5:
     sys.stdout.write('\r{}: {}\{}'.format("Fitting Models on Balanced Samples", i, nmodels)) # 第几个模型  
     control_sample.sample(len(treated_sample)).append(treated_sample).dropna()  # 模型选择相同数量的对照组和控制组样本
     y_samp, X_samp = patsy.dmatrices(formula, data=df, return_type='dataframe')  # 选出模型的自变量和因变量
     glm = GLM(y_samp, X_samp, family=sm.families.Binomial())  # 逻辑回归模型 
     try:
         res = glm.fit()
         preds = [1.0 if i >= .5 else 0.0 for i in res.predict(X_samp)]     
         preds=pd.DataFrame(preds)
         preds.columns=y_samp.columns
         b=y_samp.reset_index(drop=True)
         a=preds.reset_index(drop=True)
         ab_score=((a.sort_index().sort_index(axis=1) == b.sort_index().sort_index(axis=1)).sum() * 1.0 / len(y_samp)).values[0] # 模型预测准确性得分
         model_accuracy.append(ab_score) 
         models.append(res) 
         i += 1 
     except Exception as e:
         errors += 1
         print('Error: {}'.format(e))

print("\nAverage Accuracy:", "{}%". format(round(np.mean(model_accuracy) * 100, 2)))  # 所有模型的平均准确性


In [ ]:
threshold=0.001
method='min'
nmatches=1
test_scores = data_p[data_p[Y_field[0]]==True][['scores']]
ctrl_scores = data_p[data_p[Y_field[0]]==False][['scores']]
result, match_ids = [], []
for i in range(len(test_scores)):
    match_id = i
    score = test_scores.iloc[i]
    matches = abs(ctrl_scores - score).sort_values('scores').head(nmatches)
    chosen = np.random.choice(matches.index,  nmatches, replace=False)     
    result.extend([test_scores.index[i]] + list(chosen))
    match_ids.extend([i] * (len(chosen)+1))
    ctrl_scores=ctrl_scores.drop(chosen,axis=0)
matched_data =data_p.loc[result]
matched_data['match_id'] = match_ids
matched_data['record_id'] = matched_data.index
